In [30]:
%load_ext dotenv
%dotenv

In [31]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from dotenv import load_dotenv
from langchain.document_loaders import JSONLoader

In [32]:
import os
import openai
from dotenv import load_dotenv

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["DEFAULT_EMBED_BATCH_SIZE"] = "10"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [33]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

In [34]:
llm = OpenAI(engine="davinci",temperature=0)
text_splitter = CharacterTextSplitter()

                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [35]:
from langchain.document_loaders import JSONLoader
import logging, json, os
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = JSONLoader("data/MedicalRecords.json", jq_schema='.[]', text_content=False)
rawDocs = loader.load()

textSplitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = textSplitter.split_documents(rawDocs)
logging.info("Docs " + str(len(docs)))

In [36]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

In [37]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [38]:

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="gpt-35-turbo")

In [39]:
from langchain.chains.summarize import load_summarize_chain

In [40]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(temperature=0.7, engine="gpt-4")
# llm=ChatOpenAI(temperature=0.7, engine="gpt-4", max_tokens=1000)

                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [41]:
prompt_template = """You are an AI assistant tasked with summarizing medical records documents. Your summary should accurately capture the key information in the document while avoiding the omission of any domain-specific words. Please generate a concise and comprehensive summary that includes details such as patient information, medical history, allergies, chronic conditions, previous surgeries, prescribed medications, and upcoming appointments. Ensure that the summary is easy to understand for healthcare professionals and provides an accurate representation of the patient's medical history and current health status. Begin the summary with a brief introduction of the patient, followed by the main points of their medical records. Please remember to use clear language and maintain the integrity of the original information without missing any important details:


{text}

"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="map_reduce", return_intermediate_steps=False, map_prompt=PROMPT, combine_prompt=PROMPT)
# chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True)
chain({"input_documents": docs}, return_only_outputs=True)

{'output_text': 'Patient Summary:\n\nJohn Doe is a 41-year-old male with a history of hypertension, type 2 diabetes, and hyperlipidemia. He has allergies to penicillin, causing rash and difficulty breathing, and peanuts, resulting in anaphylaxis. John has undergone an appendectomy and a knee arthroscopy, the latter leading to infection. His current medications are Metformin, Lisinopril, and Atorvastatin. He is also taking Ibuprofen for severe lower back pain and undergoing physical therapy. John has appointments scheduled with his primary care physician and an endocrinologist for follow-ups, and he has been referred to an orthopedic specialist.\n\nKey Points:\n- Patient: John Doe, 41-year-old male\n- Medical History: Hypertension, type 2 diabetes, hyperlipidemia, lower back pain\n- Allergies: Penicillin (causes rash and difficulty breathing), Peanuts (causes anaphylaxis)\n- Previous Surgeries: Appendectomy, knee arthroscopy (resulted in infection)\n- Medications: Metformin, Lisinopril,